In [1]:
# import nibabel as nib
# import numpy as np
# import torch
# from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
# from torchvision import transforms, models
# import os
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# import pandas as pd
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# # Define paths to your dataset
# data_dir = "/kaggle/input/all-zendo-dataset/DATA/"
# t2wi_dir = os.path.join(data_dir, "T2WI")
# csv_path = "/kaggle/input/all-zendo-dataset/DATA/all_centers_combined.csv"

# # Load the CSV file
# df = pd.read_csv(csv_path)
# df['image_name'] = df['image_name'].str.replace('.nii.gz', '.nii')
# print("CSV Columns:", df.columns)
# print("First few rows of CSV:")
# print(df.head())

# # Function to load a NIfTI file
# def load_nifti(file_path):
#     nifti = nib.load(file_path)
#     return nifti.get_fdata()

# # Pair T2WI images with their labels from the CSV
# image_label_pairs = []
# for t2wi_subfolder in os.listdir(t2wi_dir):
#     subfolder_path = os.path.join(t2wi_dir, t2wi_subfolder)
#     if not os.path.isdir(subfolder_path):
#         continue
#     t2wi_files = [f for f in os.listdir(subfolder_path) if f.endswith(".nii")]
#     if t2wi_files:
#         t2wi_path = os.path.join(subfolder_path, t2wi_files[0])
#         t2wi_id = t2wi_subfolder.split(".")[0]
#         label_row = df[df['image_name'].str.replace('.nii', '') == t2wi_id]
#         if not label_row.empty:
#             label = label_row['label'].iloc[0]
#             image_label_pairs.append((t2wi_path, label))
#         else:
#             print(f"No label found for T2WI ID {t2wi_id}")

# print("Paired T2WI-Label pairs:", len(image_label_pairs))
# for t2wi_path, label in image_label_pairs[:5]:
#     print(f"T2WI: {t2wi_path}, Label: {label}")

# # Compute class weights to handle imbalance
# labels = [pair[1] for pair in image_label_pairs]
# nmbic_count = labels.count(0)
# mbic_count = labels.count(1)
# total = nmbic_count + mbic_count
# class_weights = torch.tensor([total / (2 * nmbic_count), total / (2 * mbic_count)]).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
# print(f"Class Weights (NMBIC, MBIC): {class_weights}")

# # Define data augmentation and transforms
# train_transform = A.Compose([
#     A.Resize(256, 256),
#     A.HorizontalFlip(p=0.5),
#     A.VerticalFlip(p=0.5),
#     A.Rotate(limit=30, p=0.5),
#     A.RandomBrightnessContrast(p=0.3),
#     A.Normalize(mean=0.5, std=0.5),
#     ToTensorV2(),
# ])

# val_transform = A.Compose([
#     A.Resize(256, 256),
#     A.Normalize(mean=0.5, std=0.5),
#     ToTensorV2(),
# ])

# # Custom Dataset class for classification
# class BladderCancerDataset(Dataset):
#     def __init__(self, image_label_pairs, transform=None):
#         self.image_label_pairs = image_label_pairs
#         self.transform = transform
#         self.slices = []
#         for idx, (img_path, label) in enumerate(self.image_label_pairs):
#             try:
#                 img = load_nifti(img_path)
#                 img = (img - img.min()) / (img.max() - img.min() + 1e-8)
#                 middle_slice = img.shape[2] // 2
#                 self.slices.append((idx, middle_slice, img, label))
#             except Exception as e:
#                 print(f"Error loading {img_path}: {e}")

#     def __len__(self):
#         return len(self.slices)

#     def __getitem__(self, idx):
#         img_idx, z, img, label = self.slices[idx]
#         img_slice = img[:, :, z]
#         if self.transform:
#             augmented = self.transform(image=img_slice)
#             img_slice = augmented['image']
#         else:
#             img_slice = torch.FloatTensor(img_slice).unsqueeze(0)
#         label = torch.LongTensor([label])
#         return img_slice, label

#     def get_pair_index(self, idx):
#         return self.slices[idx][0]

# # Create datasets with transforms
# train_dataset = BladderCancerDataset(image_label_pairs, transform=train_transform)
# val_dataset = BladderCancerDataset(image_label_pairs, transform=val_transform)

# # Split at the pair level
# pair_indices = list(range(len(image_label_pairs)))
# train_pair_indices, val_pair_indices = train_test_split(pair_indices, test_size=0.2, random_state=42)
# train_indices, val_indices = [], []
# for slice_idx in range(len(train_dataset)):
#     pair_idx = train_dataset.get_pair_index(slice_idx)
#     if pair_idx in train_pair_indices:
#         train_indices.append(slice_idx)
#     elif pair_idx in val_pair_indices:
#         val_indices.append(slice_idx)

# # Create data loaders
# train_sampler = SubsetRandomSampler(train_indices)
# val_sampler = SubsetRandomSampler(val_indices)
# train_loader = DataLoader(train_dataset, batch_size=8, sampler=train_sampler)
# val_loader = DataLoader(val_dataset, batch_size=8, sampler=val_sampler)

# # Use a pretrained ResNet50 model
# model = models.resnet50(pretrained=True)
# model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model.fc = nn.Sequential(
#     nn.Linear(model.fc.in_features, 512),
#     nn.ReLU(),
#     nn.Dropout(0.6),
#     nn.Linear(512, 2)
# )

# # Move model to device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device)

# # Define loss, optimizer, and scheduler with weight decay
# criterion = nn.CrossEntropyLoss(weight=class_weights)
# optimizer = optim.Adam(model.parameters(), lr=0.00005, weight_decay=1e-4)
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# # Training loop with validation and early stopping
# num_epochs = 100
# train_losses, val_losses = [], []
# train_accuracies, val_accuracies = [], []
# best_val_loss = float('inf')
# patience = 10
# early_stop_counter = 0

# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0.0
#     train_correct = 0
#     train_total = 0
#     for images_batch, labels_batch in train_loader:
#         images_batch, labels_batch = images_batch.to(device), labels_batch.to(device).squeeze(1)
#         outputs = model(images_batch)
#         loss = criterion(outputs, labels_batch)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()
#         _, predicted = torch.max(outputs, 1)
#         train_total += labels_batch.size(0)
#         train_correct += (predicted == labels_batch).sum().item()
#     train_loss /= len(train_loader)
#     train_accuracy = train_correct / train_total
#     train_losses.append(train_loss)
#     train_accuracies.append(train_accuracy)

#     model.eval()
#     val_loss = 0.0
#     val_correct = 0
#     val_total = 0
#     all_preds = []
#     all_labels = []
#     with torch.no_grad():
#         for images_batch, labels_batch in val_loader:
#             images_batch, labels_batch = images_batch.to(device), labels_batch.to(device).squeeze(1)
#             outputs = model(images_batch)
#             loss = criterion(outputs, labels_batch)
#             val_loss += loss.item()
#             _, predicted = torch.max(outputs, 1)
#             val_total += labels_batch.size(0)
#             val_correct += (predicted == labels_batch).sum().item()
#             all_preds.extend(predicted.cpu().numpy())
#             all_labels.extend(labels_batch.cpu().numpy())
#     val_loss /= len(val_loader)
#     val_accuracy = val_correct / val_total
#     val_losses.append(val_loss)
#     val_accuracies.append(val_accuracy)

#     print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
    
#     scheduler.step(val_loss)
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         early_stop_counter = 0
#         torch.save(model.state_dict(), "best_model.pth")
#     else:
#         early_stop_counter += 1
#         if early_stop_counter >= patience:
#             print("Early stopping triggered!")
#             break

# # Load the best model
# model.load_state_dict(torch.load("best_model.pth"))

# # Plot training and validation accuracy/loss
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Train Accuracy')
# plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Val Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.title('Training and Validation Accuracy')
# plt.legend()
# plt.grid()
# plt.subplot(1, 2, 2)
# plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
# plt.plot(range(1, len(val_losses) + 1), val_losses, label='Val Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()
# plt.grid()
# plt.tight_layout()
# plt.show()

# # Generate and plot confusion matrix
# cm = confusion_matrix(all_labels, all_preds)
# plt.figure(figsize=(6, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['NMBIC', 'MBIC'], yticklabels=['NMBIC', 'MBIC'])
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('Confusion Matrix')
# plt.show()

# # Visualize predictions on validation set
# model.eval()
# val_iter = iter(val_loader)
# images, labels = next(val_iter)
# images, labels = images.to(device), labels.to(device).squeeze(1)
# with torch.no_grad():
#     outputs = model(images)
#     _, preds = torch.max(outputs, 1)
# images, labels, preds = images.cpu(), labels.cpu(), preds.cpu()
# num_samples = min(15, len(images))
# plt.figure(figsize=(15, 4 * num_samples))
# for i in range(num_samples):
#     plt.subplot(num_samples, 2, 2 * i + 1)
#     plt.imshow(images[i, 0], cmap="gray")
#     plt.title(f"Label: {'MBIC' if labels[i] == 1 else 'NMBIC'}")
#     plt.axis("off")
#     plt.subplot(num_samples, 2, 2 * i + 2)
#     plt.imshow(images[i, 0], cmap="gray")
#     plt.title(f"Predicted: {'MBIC' if preds[i] == 1 else 'NMBIC'}")
#     plt.axis("off")
# plt.tight_layout()
# plt.show()

In [2]:
!pip install monai 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [ ]:
import nibabel as nib
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
from monai.transforms import Compose, Resize, RandFlip, RandRotate, NormalizeIntensity, ToTensor

# Define paths to your dataset
data_dir = "/kaggle/input/all-zendo-dataset/DATA/"
t2wi_dir = os.path.join(data_dir, "T2WI")
csv_path = "/kaggle/input/all-zendo-dataset/DATA/all_centers_combined.csv"

# Load the CSV file
df = pd.read_csv(csv_path)
df['image_name'] = df['image_name'].str.replace('.nii.gz', '.nii')
print("CSV Columns:", df.columns)
print("First few rows of CSV:")
print(df.head())

# Function to load a NIfTI file
def load_nifti(file_path):
    nifti = nib.load(file_path)
    return nifti.get_fdata()

# Pair T2WI images with their labels from the CSV
image_label_pairs = []
for t2wi_subfolder in os.listdir(t2wi_dir):
    subfolder_path = os.path.join(t2wi_dir, t2wi_subfolder)
    if not os.path.isdir(subfolder_path):
        continue
    t2wi_files = [f for f in os.listdir(subfolder_path) if f.endswith(".nii")]
    if t2wi_files:
        t2wi_path = os.path.join(subfolder_path, t2wi_files[0])
        t2wi_id = t2wi_subfolder.split(".")[0]
        label_row = df[df['image_name'].str.replace('.nii', '') == t2wi_id]
        if not label_row.empty:
            label = label_row['label'].iloc[0]
            image_label_pairs.append((t2wi_path, label))
        else:
            print(f"No label found for T2WI ID {t2wi_id}")

print("Paired T2WI-Label pairs:", len(image_label_pairs))
for t2wi_path, label in image_label_pairs[:5]:
    print(f"T2WI: {t2wi_path}, Label: {label}")

# Compute class weights to handle imbalance
labels = [pair[1] for pair in image_label_pairs]
nmbic_count = labels.count(0)
mbic_count = labels.count(1)
total = nmbic_count + mbic_count
class_weights = torch.tensor([total / (2 * nmbic_count), total / (2 * mbic_count)]).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
print(f"Class Weights (NMBIC, MBIC): {class_weights}")

# Define 3D data augmentation and transforms
train_transform = Compose([
    Resize(spatial_size=(128, 128, 128)),  # Resize to fixed 3D size
    RandFlip(prob=0.5, spatial_axis=0),    # Flip along x-axis
    RandFlip(prob=0.5, spatial_axis=1),    # Flip along y-axis
    RandRotate(range_x=30, range_y=30, range_z=30, prob=0.5),  # 3D rotation
    NormalizeIntensity(),                   # Normalize intensity
    ToTensor()                             # Convert to PyTorch tensor
])

val_transform = Compose([
    Resize(spatial_size=(128, 128, 128)),
    NormalizeIntensity(),
    ToTensor()
])

# Custom Dataset class for 3D classification
class BladderCancerDataset(Dataset):
    def __init__(self, image_label_pairs, transform=None):
        self.image_label_pairs = image_label_pairs
        self.transform = transform
        self.volumes = []
        for idx, (img_path, label) in enumerate(self.image_label_pairs):
            try:
                img = load_nifti(img_path)
                img = (img - img.min()) / (img.max() - img.min() + 1e-8)  # Normalize to [0, 1]
                self.volumes.append((idx, img, label))
            except Exception as e:
                print(f"Error loading {img_path}: {e}")

    def __len__(self):
        return len(self.volumes)

    def __getitem__(self, idx):
        img_idx, img, label = self.volumes[idx]
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.FloatTensor(img).unsqueeze(0)  # Add channel dimension: [C, D, H, W]
        label = torch.LongTensor([label])
        return img, label

    def get_pair_index(self, idx):
        return self.volumes[idx][0]

# Create datasets with transforms
train_dataset = BladderCancerDataset(image_label_pairs, transform=train_transform)
val_dataset = BladderCancerDataset(image_label_pairs, transform=val_transform)

# Split at the pair level
pair_indices = list(range(len(image_label_pairs)))
train_pair_indices, val_pair_indices = train_test_split(pair_indices, test_size=0.2, random_state=42)
train_indices, val_indices = [], []
for idx in range(len(train_dataset)):
    pair_idx = train_dataset.get_pair_index(idx)
    if pair_idx in train_pair_indices:
        train_indices.append(idx)
    elif pair_idx in val_pair_indices:
        val_indices.append(idx)

# Create data loaders with smaller batch size for 3D
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
train_loader = DataLoader(train_dataset, batch_size=2, sampler=train_sampler)
val_loader = DataLoader(val_dataset, batch_size=2, sampler=val_sampler)

# Define a simple custom 3D CNN model
class Simple3DCNN(nn.Module):
    def __init__(self):
        super(Simple3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool3d(2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        # Calculate output size after conv and pooling: 128/2/2/2 = 16
        self.fc1 = nn.Linear(64 * 16 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Simple3DCNN().to(device)

# Define loss, optimizer, and scheduler with weight decay
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.00005, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# Training loop with validation and early stopping
num_epochs = 100
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []
best_val_loss = float('inf')
patience = 10
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    for images_batch, labels_batch in train_loader:
        images_batch, labels_batch = images_batch.to(device), labels_batch.to(device).squeeze(1)
        outputs = model(images_batch)
        loss = criterion(outputs, labels_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        train_total += labels_batch.size(0)
        train_correct += (predicted == labels_batch).sum().item()
    train_loss /= len(train_loader)
    train_accuracy = train_correct / train_total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images_batch, labels_batch in val_loader:
            images_batch, labels_batch = images_batch.to(device), labels_batch.to(device).squeeze(1)
            outputs = model(images_batch)
            loss = criterion(outputs, labels_batch)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels_batch.size(0)
            val_correct += (predicted == labels_batch).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels_batch.cpu().numpy())
    val_loss /= len(val_loader)
    val_accuracy = val_correct / val_total
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
    
    scheduler.step(val_loss)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "best_3d_model.pth")
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered!")
            break

# Load the best model
model.load_state_dict(torch.load("best_3d_model.pth"))

# Plot training and validation accuracy/loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_accuracies) + 1), train_accuracies, label='Train Accuracy')
plt.plot(range(1, len(val_accuracies) + 1), val_accuracies, label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.grid()
plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid()
plt.tight_layout()
# plt.savefig('training_validation_plots.png')

# Generate and plot confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['NMBIC', 'MBIC'], yticklabels=['NMBIC', 'MBIC'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
# plt.savefig('confusion_matrix.png')

# Visualize predictions on validation set (show middle slice)
model.eval()
val_iter = iter(val_loader)
images, labels = next(val_iter)
images, labels = images.to(device), labels.to(device).squeeze(1)
with torch.no_grad():
    outputs = model(images)
    _, preds = torch.max(outputs, 1)
images, labels, preds = images.cpu(), labels.cpu(), preds.cpu()
num_samples = min(4, len(images))  # Reduced due to memory/display constraints
plt.figure(figsize=(15, 4 * num_samples))
for i in range(num_samples):
    middle_slice = images[i, 0, images.shape[2] // 2]  # Middle slice along depth
    plt.subplot(num_samples, 2, 2 * i + 1)
    plt.imshow(middle_slice, cmap="gray")
    plt.title(f"Label: {'MBIC' if labels[i] == 1 else 'NMBIC'}")
    plt.axis("off")
    plt.subplot(num_samples, 2, 2 * i + 2)
    plt.imshow(middle_slice, cmap="gray")
    plt.title(f"Predicted: {'MBIC' if preds[i] == 1 else 'NMBIC'}")
    plt.axis("off")
plt.tight_layout()
# plt.savefig('prediction_visualization.png')

CSV Columns: Index(['label', 'image_name', 'mask_name', 'Age (years)', 'Gender',
       'Pathological T stage', 'Pathological grade',
       'Type of patient's tumor number'],
      dtype='object')
First few rows of CSV:
   label  image_name      mask_name  Age (years) Gender Pathological T stage  \
0      0  c1_001.nii  c1_001.nii.gz           67   Male                   Ta   
1      0  c1_002.nii  c1_002.nii.gz           58   Male                   Ta   
2      0  c1_003.nii  c1_003.nii.gz           75   Male                   Ta   
3      0  c1_004.nii  c1_004.nii.gz           77   Male                   Ta   
4      1  c1_005.nii  c1_005.nii.gz           67   Male                   T2   

  Pathological grade Type of patient's tumor number  
0                Low                         Single  
1                Low                         Single  
2                Low                         Single  
3               High                         Single  
4               High        